# 2D Advection-Diffusion equation

in this notebook we provide a simple example of the DeepMoD algorithm and apply it on the 2D advection-diffusion equation. 

In [1]:
# General imports
import numpy as np
import torch
# DeepMoD functions
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library2D_third
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold,PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

# Settings for reproducibility
np.random.seed(1)
torch.manual_seed(1)

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## Prepare the data

Next, we prepare the dataset.

In [2]:
data = loadmat('Diffusion_2D_space41.mat')
data = np.real(data['Expression1']).reshape((41,41,41,4))[:,:,:,3]
x_dim, y_dim, t_dim = data.shape

In [3]:
time_range = [1,2,4,6,8,10,12,14]

In [4]:
for i in time_range:
    
    # Downsample data and prepare data without noise:
    down_data= np.take(np.take(np.take(data,np.arange(0,x_dim,3),axis=0),np.arange(0,y_dim,3),axis=1),np.arange(0,t_dim,i),axis=2)
    print("Dowmsampled shape:",down_data.shape, "Total number of data points:", np.product(down_data.shape))
    index = len(np.arange(0,t_dim,i))    
    width, width_2, steps = down_data.shape
    x_arr, y_arr, t_arr = np.linspace(0,1,width), np.linspace(0,1,width_2), np.linspace(0,1,steps)
    x_grid, y_grid, t_grid = np.meshgrid(x_arr, y_arr, t_arr, indexing='ij')
    X, y = np.transpose((t_grid.flatten(), x_grid.flatten(), y_grid.flatten())), np.float32(down_data.reshape((down_data.size, 1)))
    
    
    # Add noise 
    noise_level = 0.60
    y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)

    # Randomize data 

    idx = np.random.permutation(y.shape[0])
    X_train = torch.tensor(X[idx, :], dtype=torch.float32, requires_grad=True).to(device)
    y_train = torch.tensor(y_noisy[idx, :], dtype=torch.float32).to(device)

    # Configure DeepMoD

    network = NN(3, [40, 40, 40, 40], 1)
    library = Library2D_third(poly_order=0) 
    estimator = Threshold(0.05) 
    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
    constraint = LeastSquares() 
    model = DeepMoD(network, library, estimator, constraint).to(device)
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3) 
    logdir='final_runs/60_noise_x14/'+str(index)+'/'
    train(model, X_train, y_train, optimizer,sparsity_scheduler, log_dir=logdir, split=0.8, max_iterations=50000, delta=1e-6, patience=200) 


Dowmsampled shape: (14, 14, 41) Total number of data points: 8036
 49975  MSE: 4.31e-04  Reg: 9.63e-06  L1: 1.51e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (14, 14, 21) Total number of data points: 4116
 49975  MSE: 4.57e-04  Reg: 3.97e-06  L1: 1.47e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (14, 14, 11) Total number of data points: 2156
 47425  MSE: 4.93e-04  Reg: 3.28e-06  L1: 1.21e+00 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 49975  MSE: 6.00e-04  Reg: 7.52e-06  L1: 1.50e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (14, 14, 6) Total number of data points: 1176
 49975  MSE: 5.31e-04  Reg: 5.06e-06  L1: 1.31e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (14, 14, 5) Total number of data points: 980
 49975  MSE: 3.16e-03  Reg: 1.17e-06  L1: 1.96e+03 Algorithm converged. Writing model to disk.
Dowmsampled shape: (14, 14, 4) Total number of data points: 784
 49975  MSE: 6.90e-04  Reg: 1.71e-05  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (14, 14, 3) Total number of data points: 588
 49975  MSE: 2.92e-03  Reg: 2.26e-05  L1: 2.54e+00 Algorithm converged. Writing model to disk.
